In [3]:
import os
import cv2
import random
import pickle
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
%matplotlib inline


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def remove_dir(path):
    if os.path.exists(path):
        os.removedirs(path)

In [2]:
dataset_dir = "D:/Datasets/Face_Mask"
train_dir = os.path.join(dataset_dir, "train")
test_dir = os.path.join(dataset_dir, "test")

create_dir(train_dir)
create_dir(test_dir)

NameError: name 'create_dir' is not defined

In [4]:
for i in os.listdir(dataset_dir):
    if i.startswith('new'):
        files_dir = os.path.join(dataset_dir, i)
        files = os.listdir(files_dir)
        random.shuffle(files)
        create_dir(os.path.join(train_dir, i))
        create_dir(os.path.join(test_dir, i))

        for j in files[:int(len(files) * 0.8)]:
            src_path = os.path.join(files_dir, j)
            dest_path = os.path.join(train_dir, i, j)
            os.rename(src_path, dest_path)
        for j in files[int(len(files) * 0.8):]:
            src_path = os.path.join(files_dir, j)
            dest_path = os.path.join(test_dir, i, j)
            os.rename(src_path, dest_path)

        remove_dir(files_dir)

In [5]:
IMG_SIZE = 224
BATCH_SIZE = 32
SEED = 42

In [6]:
train_gen = ImageDataGenerator(
    rotation_range = 20,
    validation_split = 0.2,
    horizontal_flip = True,
    preprocessing_function = tf.keras.applications.resnet50.preprocess_input
)

test_gen = ImageDataGenerator(
    preprocessing_function = tf.keras.applications.resnet50.preprocess_input
)

In [7]:
train_batch = train_gen.flow_from_directory(
    directory = train_dir,
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size = BATCH_SIZE,
    class_mode = 'sparse',
    color_mode='rgb',
    subset = 'training',
    shuffle=True,
    seed = SEED
)
valid_batch = train_gen.flow_from_directory(
    directory = train_dir,
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size = BATCH_SIZE,
    class_mode = 'sparse',
    color_mode='rgb',
    subset = 'validation',
    seed = SEED
)
test_batch = test_gen.flow_from_directory(
    directory = test_dir,
    target_size = (IMG_SIZE, IMG_SIZE),
    batch_size = BATCH_SIZE,
    class_mode = 'sparse',
    color_mode='rgb',
    seed = SEED
)

Found 9600 images belonging to 2 classes.
Found 2400 images belonging to 2 classes.
Found 4000 images belonging to 2 classes.


In [8]:
img_shape = (IMG_SIZE, IMG_SIZE, 3)
base_resnet = tf.keras.applications.resnet50.ResNet50(input_shape=img_shape, include_top=False,  weights='imagenet', pooling='avg')
print(base_resnet.summary())

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [10]:
input = base_resnet.input
x = base_resnet.output
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.5)(x)
output = layers.Dense(2, activation='softmax')(x)

model = keras.Model(inputs=input, outputs=output)
print(model.summary())


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 average_pooling2d (AverageP  (None, 4, 4, 2048)       0         
 ooling2D)                                                       
                                                                 
 dropout (Dropout)           (None, 4, 4, 2048)        0         
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 256)               8388864   
                                                             

In [11]:
model.compile(
    optimizer = keras.optimizers.Adam(learning_rate=0.0001),
    loss = keras.losses.SparseCategoricalCrossentropy(),
    metrics = ['accuracy']
)

In [12]:
checkpoint = ModelCheckpoint(
    filepath = 'D:/Github/CVPR/Finalterm/checkpoints/resnet50_checkpoint_{epoch:02d}.h5',
    monitor='val_loss',
    save_best_only=True,
    mode='auto',
    save_freq = 'epoch'
    )

early_stp = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, restore_best_weights=True)

In [13]:
hist = model.fit(
    train_batch,
    validation_data = valid_batch,
    epochs = 50,
    batch_size=BATCH_SIZE,
    callbacks = [checkpoint, early_stp]
)

Epoch 1/10
300/300 [==============================] - 2155s 7s/step - loss: 0.0980 - accuracy: 0.9876 - val_loss: 2.1315 - val_accuracy: 0.5429
Epoch 2/10
300/300 [==============================] - 2190s 7s/step - loss: 2.6025e-04 - accuracy: 0.9999 - val_loss: 1.0133e-08 - val_accuracy: 1.0000
Epoch 3/10
300/300 [==============================] - 2468s 8s/step - loss: 5.3731e-06 - accuracy: 1.0000 - val_loss: 3.2783e-09 - val_accuracy: 1.0000
Epoch 4/10
300/300 [==============================] - 2155s 7s/step - loss: 3.6157e-06 - accuracy: 1.0000 - val_loss: 8.6426e-09 - val_accuracy: 1.0000
Epoch 5/10
300/300 [==============================] - 2207s 7s/step - loss: 2.6902e-06 - accuracy: 1.0000 - val_loss: 6.4075e-09 - val_accuracy: 1.0000
Epoch 6/10
300/300 [==============================] - 2142s 7s/step - loss: 1.7377e-06 - accuracy: 1.0000 - val_loss: 5.2154e-09 - val_accuracy: 1.0000
Epoch 6: early stopping


In [14]:
test_loss, test_acc = model.evaluate(test_batch, verbose=1)
print('Test loss:', test_loss)
print('\nTest accuracy:', test_acc)

125/125 [==============================] - 237s 2s/step - loss: 1.9081e-06 - accuracy: 1.0000

Test accuracy: 1.0
Test loss: 1.9081066966464277e-06


In [15]:
model.save("D:/Github/CVPR/Finalterm/mask_detector.h5")

In [19]:
model = load_model("D:/Github/CVPR/Finalterm/mask_detector.h5")

In [20]:
def preprocess_f(frame):
    frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = np.expand_dims(frame, axis=0)
    frame = tf.keras.applications.resnet50.preprocess_input(frame)
    return frame

In [ ]:
capture = cv2.VideoCapture(0)

if not capture.isOpened():
    raise IOError("cannot open camera")

while(capture.isOpened()):
    ret, frame = capture.read()
    
    if ret:
        frame_copy = frame.copy()
        frame_copy = preprocess_f(frame_copy)
        preds = model.predict(frame_copy)
        label = np.argmax(preds)
        if label == 0:
            cv2.putText(frame, "Mask Detected", (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        elif label == 1:
            cv2.putText(frame, "No Mask Detected", (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (100, 100, 255), 2)
        cv2.imshow('Mask Detection', frame)

        c = cv2.waitKey(3)
        if c == 27: # ASCII of Esc
            break
    else:
        break

capture.release()
cv2.destroyAllWindows()
